$\textbf{IMPORTS}$

In [1]:
import requests
import json
import time
import pprint
import pandas as pd
from bs4 import BeautifulSoup
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

$\textbf{LOAD ALL PAGES}$, BOLIGA.DK

In [2]:
#1.Load pages boliga.dk
links = []
for page in range(1,149):
    url_boliga = f'https://www.boliga.dk/resultat?propertyType=3&page={page}&sort=daysForSale-a'
    links.append(url_boliga)

$\textbf{SCRAPE DATA FROM EACH PAGE}$, BOLIGA.DK

In [3]:
#Combinale all code above in a loop to scrape all pages
add = []
p = []
zipc = []
m_2 = []
rooms_ = []

for i in links:
    response = requests.get(i, headers={'name':'Sevda Öztürk','email':'txg194b@alumni.ku.dk'})
    soup = BeautifulSoup(response.content,"xml")
    
    #append address to list
    list_address = soup.find_all("div" ,class_ = 'w-100 mt-1 listing-address font-weight-bold text-nowrap text-truncate')
    address = [col.text.split(",",1)[0] for col in list_address]
    add.append(address)
    #convert list of lists to ONE LIST
    address_final = []
    for sublist in add:
        for item in sublist:
            address_final.append(item)
    
    #append price to list
    list_price = soup.find_all("div" , class_ ="price m-0 d-flex align-items-center")
    price = [col.text for col in list_price]
    p.append(price)
    #convert list of lists to ONE LIST
    price_final = []
    for sublist in p:
        for item in sublist:
            price_final.append(item)
    
    #append zipcode to list
    list_zipcode = soup.find_all("div" ,class_ = "zip-color text-nowrap text-truncate")
    zipcode = [col.text for col in list_zipcode]
    zipc.append(zipcode)
     #convert list of lists to ONE LIST
    zipcode_final = []
    for sublist in zipc:
        for item in sublist:
            zipcode_final.append(item)
    
    #append m2 to list
    list_m2 = soup.find_all("span" ,class_ = "text-nowrap")
    m2 = [col.text.split() for col in list_m2]
    #the variable m2 is a list of lists. The following code keeps the first element of every sublist to get rid of the string "m2"
    m2 = [element[0] for element in m2]
    del m2[0] #delete first element in m2 in order to get all m2 in next code
    m2 = m2[::6] #we now keep every 6th element, which is the size of every address
    m_2.append(m2)
    #convert list of lists to ONE LIST
    m2_final = []
    for sublist in m_2:
        for item in sublist:
            m2_final.append(item)
    
    #append room to list
    rooms = [col.text.split() for col in list_m2]
    rooms = [elements[1:] for elements in rooms]
    rooms = rooms[::6] #we now keep every 6th element, which is no. of rooms
    rooms_.append(rooms)
    #convert list of lists to ONE LIST
    rooms_fin = []
    for sublist in rooms_:
        for item in sublist:
            rooms_fin.append(item)
    rooms_final = []
    for sublist in rooms_fin:
        for item in sublist:
            rooms_final.append(item)
    #replace "værelse" with "1"
    for i in range(len(rooms_final)):
        if rooms_final[i] == "værelse":
            rooms_final[i] = "1"
    time.sleep(0.5) #Sleep for 0.5 seconds


$\textbf{COMBINE ALL CODE ABOVE INTO ONE DATAFRAME}$

In [4]:
#1. Combine all scraped lists in one dataframe
df_addzip = {"Pris": price_final, "address": address_final, "zipcode": zipcode_final, "m2": m2_final, "rooms": rooms_final}
df_data = pd.DataFrame(df_addzip)
df_data

,Pris,address,zipcode,m2,rooms
0,265.000kr.,Nørmarkvej 29 1 12,7600 Struer,57,2
1,475.000kr.,Skelvangsvej 105,8920 Randers NV,67,2
2,495.000kr.,Lyksborgvej 38,7500 Holstebro,43,2
3,495.000kr.,Nyvangsvej 18,8900 Randers C,50,2
4,525.000kr.,Dalgas Alle 109,7800 Skive,47,1
...,...,...,...,...,...
7267,498.000kr.,Gl. Ålborgvej 4,9300 Sæby,82,3
7268,600.000kr.,Chr. Møllers Vej 6D,9990 Skagen,72,3
7269,390.000kr.,Ringgade 82,7600 Struer,66,2
7270,595.000kr.,Tranevej 108,9990 Skagen,74,3


$\textbf{GET  DISTANCES}, \ dataforsyningen.dk, \ hvorlangterder.dk$

In [5]:
#1. Load pages
Address_zipcode = df_data["address"] + "" + df_data["zipcode"]
add_zip = []
for i in Address_zipcode:
    response_dist = requests.get(f"https://api.dataforsyningen.dk/adresser?q={i}&struktur=mini", headers={'name':'Sevda Öztürk','email':'txg194b@alumni.ku.dk'})
    res_j = response_dist.json()
    add_zip.append(res_j)

Before continuing, locate missing values due to typing error in column address in df_data

In [6]:
#2. Locate missing values
df_add_zip = pd.DataFrame(add_zip)
index = df_add_zip[df_add_zip[0].isnull()][0]

#3. Store index numbers in order to del the corresponding indexes in df_data
index_list = list(index.index.values)

#4. Delete corresponding rows in df_data
for i in index_list:
    df_add_zip.drop(i, inplace=True)

#5. reset_index keeps failing, so here comes another index-method    
df_add_zip["new_index"] = range(len(df_add_zip))
df_add_zip.set_index("new_index", inplace=True)

#vi mister 7254 - 7114 observationer efter fjernelse af missing values!!

Drop missing values in df_data

In [7]:
#6. Delete corresponding rows in df_data
for i in index_list:
    df_data.drop(i, inplace=True)

#reset_index keeps failing, so here comes another index-method    
df_data["new_index"] = range(len(df_data))
df_data.set_index("new_index", inplace=True)

#vi mister 7254 - 7114 observationer efter fjernelse af missing values!!

In [8]:
#7. Get longitude, x, and latitudes, y from address i
x = []
y = []

for i in range(len(df_add_zip)):
    xx = df_add_zip[0][i]["x"]
    x.append(xx)

for i in range(len(df_add_zip)):
    yy = df_add_zip[0][i]["y"]
    y.append(yy)

In [9]:
#8. Add coordinates to df_data
df_data["x"] = x
df_data["y"] = y

In [10]:
#3 Get distance to nearest daycare
xy = df_data["y"].apply(str) +","+ df_data["x"].apply(str)
daycare_distance = []

for i in xy:
    afstand_daycare_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={i}&poitypes=daycare&decodepolyline')
    afstand_daycare_json.json()
    key = afstand_daycare_json.json()['daycare']
    key = key['routedmeters']
    daycare_distance.append(key)

In [11]:
regionaltog_distance = []

for i in xy:
    afstand_regionaltog_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={i}&poitypes=train&decodepolyline')
    afstand_regionaltog_json.json()
    key = afstand_regionaltog_json.json()['train']
    key = key['routedmeters']
    regionaltog_distance.append(key)

In [12]:
school_distance = []

for i in xy:
    afstand_school_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={i}&poitypes=school&decodepolyline')
    afstand_school_json.json()
    key = afstand_school_json.json()['school']
    key = key['routedmeters']
    school_distance.append(key)

In [13]:
supermarket_distance = []

for i in xy:
    afstand_supermarket_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={i}&poitypes=supermarket&decodepolyline')
    afstand_supermarket_json.json()
    key = afstand_supermarket_json.json()['supermarket']
    key = key['routedmeters']
    supermarket_distance.append(key)

In [14]:
hospital_distance = []

for i in xy:
    hospital_distance_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={i}&poitypes=hospital&decodepolyline')
    hospital_distance_json.json()
    key = hospital_distance_json.json()['hospital']
    key = key['routedmeters']
    hospital_distance.append(key)

$\textbf{FINAL DATASET}$

In [16]:
#4 Add coordinates of addresses in dataframe
df_data["daycare_distance"] = daycare_distance
df_data["train_distance"] = regionaltog_distance
df_data["school_distance"] = school_distance
df_data["supermarket_distance"] = supermarket_distance
df_data["hospital_distance"] = hospital_distance
df_data

,Pris,address,zipcode,m2,rooms,x,y,daycare_distance,train_distance,school_distance,supermarket_distance,hospital_distance
new_index,,,,,,,,,,,,
0,265.000kr.,Nørmarkvej 29 1 12,7600 Struer,57,2,8.595832,56.482711,464.56,1594.73,1169.06,497.99,15116.95
1,749.000kr.,Willemoesgade 7,9000 Aalborg,44,1,9.899679,57.055060,661.02,914.89,664.37,300.57,1257.92
2,795.000kr.,Kirstinelundsparken 7,4930 Maribo,60,2,11.511325,54.775939,607.82,1409.96,306.71,317.66,1425.18
3,999.000kr.,Willemoesgade 7,9000 Aalborg,65,2,9.899679,57.055060,661.02,914.89,664.37,300.57,1257.92
4,1.500.000kr.,Banegårdsplads 5B,8382 Hinnerup,81,2,10.060419,56.262914,841.64,36.44,875.08,296.09,11954.96
...,...,...,...,...,...,...,...,...,...,...,...,...
1100,499.000kr.,Solvangsvej 7A,9640 Farsø,89,4,9.345420,56.775994,1016.47,35519.00,986.44,673.97,973.02
1101,845.000kr.,Solbakkevej 15,9230 Svenstrup J,91,5,9.852043,56.975926,335.30,582.46,842.29,477.49,8372.11
1102,499.000kr.,Skelvej 6B,8981 Spentrup,54,2,10.041683,56.537981,421.30,11179.46,653.22,427.90,8737.76
